# Object placement using Point Cloud Library

In [1]:
from typing import Sequence
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import IPython
import scipy.spatial
import time
start_time = time.time()
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d

/home/ananya/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Lucy is equipped with an RGB-D sensor (Asus Xtion) with which it obtains the required data for carrying out many of its tasks. RGB-D sensors can be used to obtain both a regular color image of a scene and what is called a depth image, in which each pixel encodes distances to the camera's sensor; these distances are typically given with respect to a reference frame whose origin is at the location of the camera's sensor.

The images depicted below are the colored image and a representation of the depth image of the dinner table at the @Home Lab taken by Lucy. 

![Lucy RGB](img/RGB_view.jpeg) 

![Lucy Depth](img/depth.jpeg)


The picture below shows the scene as a point cloud obtained by the sensor mounted on the office's ceiling. Note that the unit of distance in the point cloud is **meters**.
![Lucy PointCloud](img/point_cloud.jpeg)

This project is created to identify possible drop points where the robot could place an object it is holding.

The pose of the camera relative to Lucy's base is located $(-0.001$, $0.018$, $0.990)$ meters away from the base frame and is rotated $-107.715^o$ about $z$, $-0.626$ about $y$ and $-90.138^o$ about $x$.

## transforming the points with respect to the base of Lucy.

In [2]:
# Loading point cloud data. 
point_cloud = np.genfromtxt("data/cloud.pcd", skip_header=11)
transformed_points = None

# Returns a matrix of a frame rotated about X
def get_RX(theta: float) -> np.ndarray:
    A_B_R = np.array([[1., 0.,             0.], 
                      [0., np.cos(theta), -np.sin(theta)], 
                      [0., np.sin(theta), np.cos(theta)]])
    return A_B_R

def get_RY(theta: float) -> np.ndarray:
    A_B_R = np.array([[np.cos(theta),  0., np.sin(theta)], 
                      [0.,             1., 0.], 
                      [-np.sin(theta), 0., np.cos(theta)]])
    return A_B_R
    
def get_RZ(theta: float) -> np.ndarray:
    A_B_R = np.array([[np.cos(theta), -np.sin(theta), 0.], 
                      [np.sin(theta), np.cos(theta),  0.], 
                      [0.,            0.,             1.]])
    return A_B_R

# rotation_angles: Sequence[float] -- rotation angles (in degrees) in the order [z, y, x]
# translation: Sequence[float] -- translation (in meters) in the order [x, y, z]
def get_homogeneous_transform(rotation_angles: Sequence[float],
                              translation: Sequence[float]) -> np.ndarray:
    
    rotation_matrix = np.dot(np.dot(get_RZ(np.deg2rad(rotation_angles[0])), get_RY(np.deg2rad(rotation_angles[1]))), get_RX(np.deg2rad(rotation_angles[2])))
    translation = translation[np.newaxis].T
    T = np.hstack((rotation_matrix, translation))
    T = np.vstack((T, np.array([0., 0., 0., 1.])))    
    return T

T_C_B = get_homogeneous_transform(np.array([-107.715, -0.626, -90.138]), np.array([-0.001 , 0.018, 0.990]))

temp_matrix = np.array([[0.],
                       [0.],
                       [0.],
                       [0.]])

for i in range(point_cloud.shape[0]):
    point_cloud_vector = np.vstack((point_cloud[i,:][np.newaxis].T, np.array([1.])))
    output = np.dot(T_C_B, point_cloud_vector)
    temp_matrix = np.hstack((temp_matrix, output))

transformed_points = np.delete(temp_matrix, 0, 1)
transformed_points = np.delete(transformed_points, (3), axis=0)
transformed_points = transformed_points.T


## implementing the point ranking algorithm by filling out the missing code in the function below.

In [3]:
def rate_placements(point_cloud: np.ndarray) -> np.ndarray:
    """
    This function returns the angles of each point in point_cloud
    with respect to the Z-axis.
    """
    point_tree = scipy.spatial.cKDTree(point_cloud)
    angles = np.array([])
    cnt = 0
    for a_point in point_cloud[:,:]:
        # this function call returns index of all the points that are at a distance of 0.04 from point a_point
        results_idx = point_tree.query_ball_point(a_point, 0.04, p=2) # r=0.04 (in meter), and p=2 (L2 norm)
        if len(results_idx) == 1:
            angle = 1.0
            angles = np.hstack((angles, angle))
            continue
        
        nearby_points = point_cloud[results_idx]        
        cov = np.cov(nearby_points.T)
        eigenvalues, eigenvectors = np.linalg.eigh(cov)
        #taking min eigenvalue's eigenvector
        w = eigenvectors[:,0]
        #Finding a angle between normal vector(w) and Z vector
        w = w.ravel()
        z_vector = [0,0,1]
        unit_vector1 = w / np.linalg.norm(w)
        unit_vector2 = z_vector / np.linalg.norm(z_vector)
        dot_product = np.dot(unit_vector1, unit_vector2)
        angle = np.arccos(dot_product) #angle in radian
        angles = np.hstack((angles, angle))
    return angles

angles = rate_placements(transformed_points)

angles = np.array(angles)
angles = np.where(angles > 1 , 1, angles)

## saving the x,y,z positions with their corresponding angles

In [4]:
new_transformed_points = []
for i in range(len(transformed_points)):
    new_transformed_points.append(np.hstack((transformed_points[i], angles[i])))

## visualizing the obtained locations where the robot can place the object (red)

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')
print(len(new_transformed_points))
for i in new_transformed_points:
    x = i[0]
    y = i[1]
    z = i[2]
    if i[3] == 1:
        ax.scatter(x,y,z,color= 'red', s=2)
    else:
        ax.scatter(x,y,z,color= 'blue', s=2)

ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')
fig.savefig('plot.png')

![result](plot.png)

![Lucy RGB](plot.png) 